## Sentiment Analysis with Amazon SageMaker AutoPilot
Using AutoPilot to Predict Customer Review Sentiment

## Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  AutoPilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and sentiment classification machine learning pipeline.

## Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

In [ ]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [ ]:
# Unbalanced
#prefix_unbalanced_raw_train = 'feature-store/amazon-reviews-autopilot/raw-labeled-split-unbalanced-header-train-csv'

# Balanced
prefix_balanced_raw_train = 'feature-store/amazon-reviews-autopilot/raw-labeled-split-balanced-header-train-csv'

balanced_raw_with_header_train_s3_uri = 's3://{}/{}/data.csv'.format(bucket, prefix_balanced_raw_train)

print(balanced_raw_with_header_train_s3_uri)

In [ ]:
!aws s3 ls $balanced_raw_with_header_train_s3_uri

In [ ]:
prefix_output = 'models/amazon-reviews-autopilot'

autopilot_model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_output)

print(autopilot_model_output_s3_uri)


In [ ]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(balanced_raw_with_header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'is_positive_sentiment'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(autopilot_model_output_s3_uri)
}

#problem_type = 'Classification'

#auto_ml_job_objective = {
#    'MetricName': 'Accuracy'
#}

## Launching the SageMaker AutoPilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [ ]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

In [ ]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType=problem_type,
#                      AutoMLJobObjective=auto_ml_job_objective,
                      RoleArn=role)

### Tracking the progress of the AutoPilot job
SageMaker AutoPilot job consists of four high-level steps : 
* Data Preprocessing, where the dataset is split into train and validation sets.
* Recommending Pipelines, where the dataset is analyzed and SageMaker AutoPilot comes up with a list of ML pipelines that should be tried out on the dataset.
* Automatic Feature Engineering, where SageMaker AutoPilot performs feature transformation on individual features of the dataset as well as at an aggregate level.
* ML pipeline selection and hyperparameter tuning, where the top performing pipeline is selected along with the optimal hyperparameters for the training algorithm (the last stage of the pipeline). 

In [ ]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

# View Generated Notebooks
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

In [ ]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

### Let's copy all of the generated resources including the two notebooks.

In [ ]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

In [ ]:
!rm -rf ./generated_module
!rm -rf ./notebooks
!aws s3 cp --recursive $generated_resources .

## In the file view, open the `notebooks/` and `generated_module/` folders. 
Lots of useful information in these folders ^^

# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [ ]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

# Inspect Trials using Experiments API
SageMaker AutoPilot automatically creates a new experiment, and pushes information for each trial. 

In [ ]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
df

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [ ]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

In [ ]:
best_candidate

We can see the containers and models composing the Inference Pipeline.

In [ ]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

In [ ]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Let's deploy the pipeline.

In [ ]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
xgb_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(xgb_endpoint_name)
print(variant_name)

In [ ]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m4.xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [ ]:
create_endpoint_response = sm.create_endpoint(EndpointName=xgb_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

## Wait for Model to Deploy as a REST Endpoint
This may take 5-10 mins.  Please be patient.

In [ ]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=xgb_endpoint_name)


In [ ]:
resp = sm.describe_endpoint(EndpointName=xgb_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

# Calculate Test Metrics
Let's predict and score the test set.  We'll compute metrics ourselves just for fun.

In [ ]:
sm_runtime = boto3.client('sagemaker-runtime')

In [ ]:
# review_body
csv_line_predict_positive = """I loved it!  I will recommend this to everyone."""

# All features
#csv_line_predict_positive = """US,40105404,R1L48HCJML1LAU,B0067SK29S,999729291,Mavis Beacon Teaches Typing Deluxe - 25th Anniversary Edition SB,Digital_Software,5,0,0,N,Y,... several different typing programs and this one is the best.,I've tried several different typing programs and this one is the best.,2015-04-25"""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
# review_body
csv_line_predict_negative = """Really bad.  I hope they stop making this."""

# All features
#csv_line_predict_negative = """US,50751455,R5018NV9ZUIJR,B00NG7K4PU,945644004,TurboTax Premier Fed + Efile + State,Digital_Software,1,1,1,N,Y,Horrible download to MAC,Horrible download to MAC. Keeps saying please wait while we make sure everything is up to date. It's been an hour!,2015-03-14"""
response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

In [ ]:
# Unbalanced
# prefix_unbalanced_test_raw = 'feature-store/amazon-reviews-autopilot/raw-labeled-split-unbalanced-header-test-csv'

# Balanced
refix_balanced_test_raw = 'feature-store/amazon-reviews-autopilot/raw-labeled-split-balanced-header-test-csv'

balanced_raw_with_header_test_s3_uri = 's3://{}/{}/data.csv'.format(bucket, prefix_balanced_test_raw)

balanced_raw_with_header_test_path = './{}/data.csv'.format(prefix_balanced_test_raw)

print(balanced_raw_with_header_test_s3_uri)

In [ ]:
!aws s3 cp $balanced_raw_with_header_test_s3_uri $balanced_raw_with_header_test_path

## Load the test data

In [ ]:
def load_dataset(path, sep, header):
    data = pd.read_csv(path, sep=sep, header=header)

    labels = data.iloc[:,0]
    features = data.drop(data.columns[0], axis=1)
    
    if header==None:
        # Adjust the column names after dropped the 0th column above
        # New column names are 0 (inclusive) to len(features.columns) (exclusive)
        new_column_names = list(range(0, len(features.columns)))
        features.columns = new_column_names

    return features, labels

In [ ]:
X_test, y_test = load_dataset(path=balanced_raw_with_header_test_path, sep=',', header=0)
X_test.head(5)

In [ ]:
y_test.head(5)

In [ ]:
payload = X_test.to_csv(index=False, header=False).rstrip()

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, 
                                                  Body=payload.encode('utf-8'),
                                                  ContentType='text/csv')['Body'].read().decode('utf-8')

In [ ]:
import numpy as np

preds_test = np.fromstring(response, sep='\n')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix

print('Test Accuracy: ', accuracy_score(y_test, preds_test))
print('Test Precision: ', precision_score(y_test, preds_test, average=None))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

df_cm_test = confusion_matrix(y_test, preds_test)
df_cm_test

In [ ]:
import itertools

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def plot_conf_mat(cm, classes, title, cmap = plt.cm.Greens):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
        horizontalalignment="center",
        color="black" if cm[i, j] > thresh else "black")

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

# Plot non-normalized confusion matrix
plt.figure()
fig, ax = plt.subplots(figsize=(6,4))
plot_conf_mat(df_cm_test, classes=['Not Positive Sentiment', 'Positive Sentiment'], 
                          title='Confusion matrix')
plt.show()

In [ ]:
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

auc = round(metrics.roc_auc_score(y_test, preds_test), 4)
print('AUC is ' + repr(auc))

fpr, tpr, _ = metrics.roc_curve(y_test, preds_test)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b',
label='AUC = %0.2f'% auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.1])
plt.ylim([-0.1,1.1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()